In [ ]:
%pip install psycopg2
%pip install sqlalchemy

In [1]:
import psycopg2
from sqlalchemy import create_engine

# Function to create a new database
def create_database(db_name, username, password, host='localhost', port=5432):
    conn = psycopg2.connect(dbname="postgres", user=username, password=password, host=host, port=port)
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    cur.execute(f"CREATE DATABASE {db_name};")
    cur.close()
    conn.close()

# Create the database
create_database('angeliteforecast', 'postgres', 'thesis')

DuplicateDatabase: database "angeliteforecast" already exists


In [2]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

def connect_to_database(username, password, host='localhost', port=5432, db_name='angeliteforecast'):
    """Create a database connection."""
    connection_string = f'postgresql://{username}:{password}@{host}:{port}/{db_name}'
    engine = create_engine(connection_string)
    return engine
engine = connect_to_database("postgres", "thesis")



In [6]:
import pandas as pd


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

similar_majors_dict = {
    # CCJEF
    "Criminology": ["BS Criminology"], # Combine Criminology and BS Criminology

    
    # SAS
    "ABComm": ["BA Comm"],
    "CommArts": ["BA Comm"],
    "AdvPublicRel": ["BA Comm"], # Area of Specialization
    # "CommArts": "BA Comm",     # Lack of Data | Drop


    # SBA
    "Accounting": ["Accountancy"],
    "Accounting Tech": ["BSAcctgInfoSys"],
    "BSBA-BM-Marketi": ["BusMgt", "BSBA-MktgMgmt"],
    "BussMgmt-HRM": ["BusMgt", "BSBA-HRMgmt"],


    # SEA
    "ECE": ["ELECENG"],
    # "BSCE-CEM": ["CE", "BSCE-CEM"],
    # "BSCE-SE": ["CE", "BSCE-SE"],
    # "BSCE-TE": ["CE", "BSCE-TE"],




    # SED
    "BEED-MajSPED": ["BSNEd"],           # Name change?
    "BEED-SpecialEdu": ["BSNEd"],        # Name change?
    "BPE-SPE": ["BPEd"],                   # Name change?
    "BSED-ValEd": ["BSED-RelValEd"],
    "BSMath": ["Math"],
    

    # SHTM
    "BS EventMgmt": ["BSTM-Events"],


    # SOC
    "BSCSSysDev": ["BSCompsci"],              # New Curriculum
    "BSITAnimation": ["BSEMC-DA"],            # New Curriculum
    "BSITAreaAnimati": ["BSEMC-DA"],          # New Curriculum
    "BSCyberplusPSM": ["BSCybersecurity"],    # Same Major but with Professional Science Master's Degree
    # "BSInfoTech": "BSIT",                 # General IT Major | Lack of Data | Drop
    # "BSITMultiTec": "BSIT",               # Discontinued | Lack of Data | Drop
    "BSITAreaNetAdmi": ["BSITNetAdmin"], 
    "BSITAreaWebDev": ["BSITWebdev"],

    }

# Dictionary for major with incorrect department
incorrect_department_dict = {
    "BSBA-HRM": "SBA",
    # "MAPEH-BSED": "SED",
}

department_dict = {
    "CHTM" : "SHTM",
    "CICT" : "SOC",
}

# List of majors to drop
drop_majors = [
    # MA
    # "MAPEH-BSED",     # Lack of data | Discontinued | SED Department?
    "MBM",              # Lack of data | only 1 student
    "MSEngMgmt",        # Lack of data | only 1 student
    

    # SAS
    # "CommArts",         # Lack of data | only 1 student
    "LanguageLit",      # Lack of data | only 1 student

    # SED
    
    # SHTM
    "BSBATourism",
    "BSTM-Tourism",     # Lack of data | BSTourism? | only 2 students


    # SOC
    "AssCompSci",       # Lack of data | only 1 student
    "BSITMultiTec",     # Lack of data | Discontinued
    "BSInfoTech",       # Lack of data | only 1 student

    # SEA
    "ECETech.",        # Lack of data | Discontinued


    # Extras | Aggregate Counts
    "TOTAL",
    "GRAND TOTAL",
]



In [5]:
def clean_data(df):
    

    # Step 2: Correct departments

    for major, department in incorrect_department_dict.items():
        df.loc[df["Major"]== major, 'Department'] = department
        
    # Step 3: Rename departments
    df['Department'] = df['Department'].replace(department_dict)

    # Step 4: Drop unwanted majors
    # df = df[~df['Major'].isin(drop_majors)]

    # df = df[~df['Department'].isin(['GS', 'JHS', 'HAUSPELL', 'HAU', 'MA'])]
    df = df.reset_index(drop=True)
    df =df.sort_values(by=["Start_Year", "Semester", "Department", "Major"])
    return df


In [6]:
df = pd.read_csv("data/Enrollment_Data.csv").sort_values(by=["Start_Year", "Semester", "Department", "Major"])
cpi_df = pd.read_csv("data/CPI_Education.csv").sort_values(by=["Year", "Month"])
inflation_df = pd.read_csv("data/Inflation_Rate.csv").sort_values(by="Start_Year")
admission_df = pd.read_csv("data/Admission_Data.csv").sort_values(by=["Start_Year", "Department"])
hfce_df = pd.read_csv("data/HFCE.csv").sort_values(by=["Start_Year", "Quarter"])

df = clean_data(df)

# Upload data to database
df.to_sql('enrollment', engine, if_exists='replace', index=False)
cpi_df.to_sql('cpi_education', engine, if_exists='replace', index=False)
inflation_df.to_sql('inflation_rate', engine, if_exists='replace', index=False)
admission_df.to_sql('admission', engine, if_exists='replace', index=False)
hfce_df.to_sql('hfce', engine, if_exists='replace', index=False)


54

In [12]:
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.sql import func

Base = declarative_base()

# Define the association table
user_roles = Table('user_roles', Base.metadata,
    Column('user_id', Integer, ForeignKey('users.id')),
    Column('role_id', Integer, ForeignKey('roles.id'))
)

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    password_hash = Column(String(255), nullable=False)
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())

    roles = relationship('Role', secondary=user_roles, back_populates='users')

class Role(Base):
    __tablename__ = 'roles'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), unique=True, nullable=False)
    description = Column(Text)

    users = relationship('User', secondary=user_roles, back_populates='roles')

# Create the tables
Base.metadata.create_all(engine)

print("Tables for users, roles, and user_roles have been created.")

Tables for users, roles, and user_roles have been created.


In [19]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func


# Create a base class for declarative models
Base = declarative_base()

# Define the Major model
class Major(Base):
    __tablename__ = 'majors'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True)
    department = Column(String(50), nullable=False)
    latest_year = Column(Integer)  # New column for the latest year
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())

    def __repr__(self):
        return f"<Major(name='{self.name}', department='{self.department}', latest_year={self.latest_year})>"

# Create the table in the database
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# # Example: Add a new major
# new_major = Major(name="Computer Science", department="School of Computing")
# session.add(new_major)
# session.commit()

# # Example: Query all majors
# majors = session.query(Major).all()
# for major in majors:
#     print(major)

# # Don't forget to close the session when you're done
# session.close()

D:\Local\temp\ipykernel_2364\1418014355.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [20]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
import pandas as pd

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

def get_college_data(df):
    df = df[~df['Department'].isin(['GS', 'JHS', 'SHS', 'HAUSPELL', 'HAU', 'MA'])]
    df = df[~df['Major'].isin(['TOTAL', 'GRAND TOTAL'])]
    df = df.drop(df.loc[:, "Grade_1":"Grade_12"].columns, axis=1)
    df = df.reset_index(drop=True)
    return df

def clean_college_data(df):
    # Step 1: Replace similar majors
    for old_majors, new_major in similar_majors_dict.items():
        df['Major'] = df['Major'].replace(old_majors, new_major[0])
     
    # Step 4: Drop unwanted majors
    df = df[~df['Major'].isin(drop_majors)]
        
    # Step 6: Combine majors with the same name within the same semester
    df = df.groupby(['Start_Year', 'Semester',
                     'Major', 'Department'], as_index=False).sum(["1st_Year", "2nd_Year", "3rd_Year",
                                                                  "4th_Year", "5th_Year", "TOTAL"])
    return df

# Load the enrollment data
enrollment_df = pd.read_sql_table('enrollment', engine)

# Apply the get_college_data function
college_data = get_college_data(enrollment_df)
college_data = clean_college_data(college_data)

# Get unique combinations of Major and Department with the latest year
unique_majors = college_data.groupby(['Major', 'Department'])['Start_Year'].max().reset_index()

# Get the list of existing majors from the database
existing_majors = session.query(Major.name, Major.latest_year).all()
existing_majors_dict = {major[0]: major[1] for major in existing_majors}

# Add or update each unique Major-Department combination in the majors table
new_majors = []
updated_majors = []
for _, row in unique_majors.iterrows():
    if row['Major'] not in existing_majors_dict:
        new_majors.append(Major(name=row['Major'], department=row['Department'], latest_year=row['Start_Year']))
        print(f"Adding new major: {row['Major']} - {row['Department']} (Year: {row['Start_Year']})")
    elif row['Start_Year'] > existing_majors_dict[row['Major']]:
        major = session.query(Major).filter_by(name=row['Major']).first()
        major.latest_year = row['Start_Year']
        updated_majors.append(major)
        print(f"Updating major: {row['Major']} - New latest year: {row['Start_Year']}")
    else:
        print(f"Major already up to date: {row['Major']} (Year: {existing_majors_dict[row['Major']]})")

# Bulk insert new majors and update existing ones
if new_majors:
    session.bulk_save_objects(new_majors)
    print(f"Added {len(new_majors)} new majors to the database.")
if updated_majors:
    session.bulk_save_objects(updated_majors, update_changed_only=True)
    print(f"Updated {len(updated_majors)} existing majors in the database.")

# Commit the changes
session.commit()

# Close the session
session.close()

print("Major update process completed.")

D:\Local\temp\ipykernel_2364\2633483908.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


ProgrammingError: (psycopg2.errors.UndefinedColumn) column majors.latest_year does not exist
LINE 1: SELECT majors.name AS majors_name, majors.latest_year AS maj...
                                           ^

[SQL: SELECT majors.name AS majors_name, majors.latest_year AS majors_latest_year 
FROM majors]
(Background on this error at: https://sqlalche.me/e/20/f405)

# CAUTION

In [ ]:
# Throw error to avoid run all to here
raise Exception("Execution stopped to prevent running the irreversible code.")


## Clear majors Table

In [9]:
from sqlalchemy import text

# Create a new session
Session = sessionmaker(bind=engine)
session = Session()

try:
    # Delete all rows from the majors table
    session.execute(text("DELETE FROM majors"))
    
    # Commit the transaction
    session.commit()
    print("All rows have been deleted from the majors table.")
except Exception as e:
    # If an error occurs, rollback the changes
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    # Close the session
    session.close()

All rows have been deleted from the majors table.


## Delete majors Table

In [18]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Create a MetaData instance
metadata = MetaData()

# Reflect the existing tables
metadata.reflect(bind=engine)


try:
    # Check if the majors table exists
    if 'majors' in metadata.tables:
        # Drop the majors table
        metadata.tables['majors'].drop(engine)
        print("The 'majors' table has been successfully dropped.")
    else:
        print("The 'majors' table does not exist.")
    
    # Commit the transaction
    session.commit()
except Exception as e:
    # If an error occurs, rollback the changes
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    # Close the session
    session.close()

print("Table deletion process completed.")

The 'majors' table has been successfully dropped.
Table deletion process completed.
